# BRONZE DATA LOAD

In [8]:
import pandas as pd
import psycopg2
import os
import time

In [ ]:
# DATAWAREHOUSE CONFIGURATION
DB_CONFIG = {
    "host": "localhost",        # Postgres host
    "port": 5432,               # default port
    "dbname": "DataWarehouse",            # Database
    "user": "userhere",          # DB user
    "password": "passwordhere"       # DB user password
}

In [14]:
# DATA SOURCES

DATASETS_DIR = "../../datasets"  # Path to datasets directory

TABLES_AND_FILES = {
    "bronze.crm_cust_info": "source_crm/cust_info.csv",
    "bronze.crm_prd_info": "source_crm/prd_info.csv",
    "bronze.crm_sales_details": "source_crm/sales_details.csv",
    "bronze.erp_loc_a101": "source_erp/LOC_A101.csv",
    "bronze.erp_cust_az12": "source_erp/CUST_AZ12.csv",
    "bronze.erp_px_cat_g1v2": "source_erp/PX_CAT_G1V2.csv"
}

### CRM

In [16]:
# DATA EXPLORATION
cust_info_df = pd.read_csv(os.path.join(DATASETS_DIR, TABLES_AND_FILES["bronze.crm_cust_info"]))
cust_info_df.info()
cust_info_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18494 entries, 0 to 18493
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cst_id              18490 non-null  float64
 1   cst_key             18494 non-null  object 
 2   cst_firstname       18486 non-null  object 
 3   cst_lastname        18487 non-null  object 
 4   cst_marital_status  18487 non-null  object 
 5   cst_gndr            13916 non-null  object 
 6   cst_create_date     18490 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1011.5+ KB


,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
0,11000.0,AW00011000,Jon,Yang,M,M,2025-10-06
1,11001.0,AW00011001,Eugene,Huang,S,M,2025-10-06
2,11002.0,AW00011002,Ruben,Torres,M,M,2025-10-06
3,11003.0,AW00011003,Christy,Zhu,S,F,2025-10-06
4,11004.0,AW00011004,Elizabeth,Johnson,S,F,2025-10-06


In [20]:
prd_info_df = pd.read_csv(os.path.join(DATASETS_DIR, TABLES_AND_FILES["bronze.crm_prd_info"]))
prd_info_df.info()
prd_info_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   prd_id        397 non-null    int64  
 1   prd_key       397 non-null    object 
 2   prd_nm        397 non-null    object 
 3   prd_cost      395 non-null    float64
 4   prd_line      380 non-null    object 
 5   prd_start_dt  397 non-null    object 
 6   prd_end_dt    200 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 21.8+ KB


,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
0,210,CO-RF-FR-R92B-58,HL Road Frame - Black- 58,NaN,R,2003-07-01,NaN
1,211,CO-RF-FR-R92R-58,HL Road Frame - Red- 58,NaN,R,2003-07-01,NaN
2,212,AC-HE-HL-U509-R,Sport-100 Helmet- Red,12.0,S,2011-07-01,2007-12-28
3,213,AC-HE-HL-U509-R,Sport-100 Helmet- Red,14.0,S,2012-07-01,2008-12-27
4,214,AC-HE-HL-U509-R,Sport-100 Helmet- Red,13.0,S,2013-07-01,NaN


In [21]:
sales_details_df = pd.read_csv(os.path.join(DATASETS_DIR, TABLES_AND_FILES["bronze.crm_sales_details"]))
sales_details_df.info()
sales_details_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sls_ord_num   60398 non-null  object 
 1   sls_prd_key   60398 non-null  object 
 2   sls_cust_id   60398 non-null  int64  
 3   sls_order_dt  60398 non-null  int64  
 4   sls_ship_dt   60398 non-null  int64  
 5   sls_due_dt    60398 non-null  int64  
 6   sls_sales     60390 non-null  float64
 7   sls_quantity  60398 non-null  int64  
 8   sls_price     60391 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 4.1+ MB


,sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price
0,SO43697,BK-R93R-62,21768,20101229,20110105,20110110,3578.0,1,3578.0
1,SO43698,BK-M82S-44,28389,20101229,20110105,20110110,3400.0,1,3400.0
2,SO43699,BK-M82S-44,25863,20101229,20110105,20110110,3400.0,1,3400.0
3,SO43700,BK-R50B-62,14501,20101229,20110105,20110110,699.0,1,699.0
4,SO43701,BK-M82S-44,11003,20101229,20110105,20110110,3400.0,1,3400.0


### ERP

In [22]:
cust_az12_df = pd.read_csv(os.path.join(DATASETS_DIR, TABLES_AND_FILES["bronze.erp_cust_az12"]))
cust_az12_df.info()
cust_az12_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CID     18484 non-null  object
 1   BDATE   18484 non-null  object
 2   GEN     17012 non-null  object
dtypes: object(3)
memory usage: 433.3+ KB


,CID,BDATE,GEN
0,NASAW00011000,1971-10-06,Male
1,NASAW00011001,1976-05-10,Male
2,NASAW00011002,1971-02-09,Male
3,NASAW00011003,1973-08-14,Female
4,NASAW00011004,1979-08-05,Female


In [23]:
cust_loc_a101_df = pd.read_csv(os.path.join(DATASETS_DIR, TABLES_AND_FILES["bronze.erp_loc_a101"]))
cust_loc_a101_df.info()
cust_loc_a101_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CID     18484 non-null  object
 1   CNTRY   18152 non-null  object
dtypes: object(2)
memory usage: 288.9+ KB


,CID,CNTRY
0,AW-00011000,Australia
1,AW-00011001,Australia
2,AW-00011002,Australia
3,AW-00011003,Australia
4,AW-00011004,Australia


In [24]:
cust_px_cat_g1v2_df = pd.read_csv(os.path.join(DATASETS_DIR, TABLES_AND_FILES["bronze.erp_px_cat_g1v2"]))
cust_px_cat_g1v2_df.info()
cust_px_cat_g1v2_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           37 non-null     object
 1   CAT          37 non-null     object
 2   SUBCAT       37 non-null     object
 3   MAINTENANCE  37 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


,ID,CAT,SUBCAT,MAINTENANCE
0,AC_BR,Accessories,Bike Racks,Yes
1,AC_BS,Accessories,Bike Stands,No
2,AC_BC,Accessories,Bottles and Cages,No
3,AC_CL,Accessories,Cleaners,Yes
4,AC_FE,Accessories,Fenders,No


## LOAD DATA

In [19]:
# ===========================
# Functions
# ===========================

def load_csv_into_table(cursor, table_name, file_path):
    """Truncate table and load CSV using COPY from Python (STDIN)."""
    print(f"Loading table {table_name} from {file_path}...")
    start_time = time.time()

    # Truncate table first
    cursor.execute(f"TRUNCATE TABLE {table_name};")

    # Open CSV file and COPY
    with open(file_path, 'r') as f:
        cursor.copy_expert(
            sql=f"COPY {table_name} FROM STDIN WITH CSV HEADER",
            file=f
        )

    end_time = time.time()
    print(f"{table_name} loaded in {end_time - start_time:.2f} seconds.\n")


# ===========================
# Main Execution
# ===========================
def main():
    total_start = time.time()
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        conn.autocommit = True
        cur = conn.cursor()

        for table, rel_file in TABLES_AND_FILES.items():
            file_path = os.path.join(DATASETS_DIR, rel_file)
            if not os.path.isfile(file_path):
                print(f"File not found: {file_path}")
                continue

            load_csv_into_table(cur, table, file_path)

        cur.close()
        conn.close()
        total_end = time.time()
        print(f"All tables loaded. Total duration: {total_end - total_start:.2f} seconds.")

    except Exception as e:
        print("ERROR during loading:", str(e))


if __name__ == "__main__":
    main()

Loading table bronze.crm_cust_info from ../../datasets/source_crm/cust_info.csv...
bronze.crm_cust_info loaded in 0.06 seconds.

Loading table bronze.crm_prd_info from ../../datasets/source_crm/prd_info.csv...
bronze.crm_prd_info loaded in 0.01 seconds.

Loading table bronze.crm_sales_details from ../../datasets/source_crm/sales_details.csv...
bronze.crm_sales_details loaded in 0.12 seconds.

Loading table bronze.erp_loc_a101 from ../../datasets/source_erp/LOC_A101.csv...
bronze.erp_loc_a101 loaded in 0.02 seconds.

Loading table bronze.erp_cust_az12 from ../../datasets/source_erp/CUST_AZ12.csv...
bronze.erp_cust_az12 loaded in 0.05 seconds.

Loading table bronze.erp_px_cat_g1v2 from ../../datasets/source_erp/PX_CAT_G1V2.csv...
bronze.erp_px_cat_g1v2 loaded in 0.00 seconds.

All tables loaded. Total duration: 0.28 seconds.
